# BP Experiments to verify theory for L-2 norm - MNIST

In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import division
from __future__ import print_function

import sys, os, gc, math
import numpy as np
from scipy.fftpack import dct,idct
from keras.datasets import mnist, fashion_mnist
from PIL import Image


sys.path.append('../')

from models.util import *


#Seed used for choosing classes, training points, and test points.
#SEED = 14
SEED=11

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
num_samples = 500
sqrt_n = 28
input_shape=(sqrt_n,sqrt_n,1)
n = sqrt_n*sqrt_n
k = 40
c=2.0

In [3]:
#Load MNIST data
(X_train, _), (X_test, _) = mnist.load_data()
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

m_data = np.concatenate((X_train,X_test))

In [4]:
#Normalize the data
m_data = m_data/255.0

In [5]:
#Check MNIST results for 1000 random images - BP
subset_idx = np.random.choice(np.arange(m_data.shape[0]),num_samples)
m_data_sub_bp = m_data[subset_idx]
m_data_y_bp = np.zeros((num_samples,sqrt_n,sqrt_n))
for i in range(num_samples):
    #first sample an element from the data
    x = m_data_sub_bp[i,:,:].flatten()
    x_hat = dct(x,norm='ortho')
    x_k = idct(get_topk_vec(x_hat,k=k),norm='ortho')
    e = np.random.uniform(size=n)
    y = x_k + e
    m_data_y_bp[i,:,:] = y.reshape((sqrt_n,sqrt_n))  
    m_data_sub_bp[i,:,:,0] = x_k.reshape((sqrt_n,sqrt_n))

# Get \delta_p and \Delta_p

In [6]:
#Form the matrix F
F = get_matrix(n,tf='dct')

In [7]:
%%capture three
#BP MNIST
errors_l2_bp = np.zeros(m_data_y_bp.shape[0])
errors_l1_bp = np.zeros(m_data_y_bp.shape[0])
eta_bp = np.zeros(m_data_y_bp.shape[0])
diff_l2_bp = np.zeros(m_data_y_bp.shape[0])
diff_l1_bp = np.zeros(m_data_y_bp.shape[0])

for i in range(num_samples):
    y = m_data_y_bp[i,:,:].flatten()
    x = m_data_sub_bp[i,:,:].flatten()
    
    #Get actual top k and bottom k
    x_hat_top_k = dct(x, norm='ortho')
    
    e = y - x 
    eta_bp[i] = np.linalg.norm(e)
    eta = eta_bp[i]

    x_hat_approx = socp(y,F.T,n=n,eta=eta)
    
    
    #Note the errors
    errors_l2_bp[i] = np.linalg.norm(x_hat_top_k.flatten()- x_hat_approx.flatten())
    errors_l1_bp[i] = np.linalg.norm(x_hat_top_k.flatten()- x_hat_approx.flatten(), ord=1)


    #Get the multiplicative constant
    c_l2 = 6*eta_bp[i]
    c_l1 = np.sqrt(k)*4*eta_bp[i]
    
    
    #Calculate the difference from the upper bound
    diff_l2_bp[i] = c_l2 - errors_l2_bp[i] 
    diff_l1_bp[i] = c_l1 - errors_l1_bp[i]

In [8]:
print(np.mean(errors_l2_bp), 
      np.mean(diff_l2_bp), 
      np.mean(errors_l1_bp), 
      np.mean(diff_l1_bp))

8.208630980438857 88.80668802848423 43.875594119381525 365.1769072086534


In [9]:
e_norms = np.zeros(m_data_y_bp.shape)
for i in range(num_samples):
    y = m_data_y_bp[i,:,:].flatten()
    x = m_data_sub_bp[i,:,:].flatten()
    e_norms[i] = np.linalg.norm(y - x)

In [10]:
np.mean(e_norms)

16.152730445789324

In [11]:
mnist_tup_bp = (m_data_y_bp, m_data_sub_bp, errors_l2_bp, errors_l1_bp, diff_l2_bp, diff_l1_bp )

import pickle
with open('data/mnist_theory_socp_l2.pickle', 'wb') as f:
    pickle.dump(mnist_tup_bp, f)
